In [ ]:
import pandas as pd
import io
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df1 = pd.read_csv(url)
df1

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164477,ZWE,Africa,Zimbabwe,2022-02-21,233571.0,219.0,281.143,5386.0,0.0,1.714,15476.302,14.511,18.628,356.874,0.000,0.114,1.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2049310.0,135.786,NaN,2971.0,0.197,0.0946,10.6,tests performed,7829096.0,4342121.0,3374371.0,112604.0,5202.0,9258.0,51.88,28.77,22.36,0.75,613.0,3172.0,0.021,50.00,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
164478,ZWE,Africa,Zimbabwe,2022-02-22,233980.0,409.0,339.571,5388.0,2.0,2.000,15503.402,27.100,22.500,357.006,0.133,0.133,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3321.0,0.220,0.1022,9.8,tests performed,7836325.0,4344922.0,3376712.0,114691.0,7229.0,9329.0,51.92,

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
states = pd.read_csv(io.BytesIO(uploaded['statelatlong.csv']))

In [ ]:
url1 = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv'
df_vac = pd.read_csv(url1)
df_vac

In [ ]:
df_vac["date"] = df_vac["date"].astype("datetime64")

In [ ]:
mydf = pd.concat([df_vac.iloc[:,0:2],df_vac.iloc[:,11]],axis=1)
mydf.dropna(inplace = True)

In [ ]:
mydf = mydf.groupby(["location"]).sum().reset_index()
#mydf['year'] = pd.DatetimeIndex(mydf['date']).year
ind = mydf[~pd.DataFrame(mydf["location"].tolist()).isin(states["City"].tolist()).any(1).values].index
newdf = mydf.drop(ind)
cleaned_df = newdf.append(mydf[mydf["location"]=="New York State"])
cleaned_df["location"].replace({"New York State": "New York"}, inplace=True)
cleaned_df.sort_values(by=["location"]).reset_index(drop=True)["location"]
add_long = cleaned_df.merge(states.iloc[:,],left_on="location",right_on="City")
add_long.drop(['City'], axis=1,inplace = True)
final_df = add_long.reindex(columns = ["Longitude","Latitude","location","daily_vaccinations"])
final_df["ratio"]= final_df['daily_vaccinations']/final_df['daily_vaccinations'].sum()

In [ ]:
import folium

# Make an empty map
m = folium.Map(location=[36,-100], tiles="OpenStreetMap", zoom_start=5)

# Show the map
m

In [ ]:
for i in range(0,len(final_df)):
   folium.Circle(
      location=[final_df.iloc[i]['Latitude'], final_df.iloc[i]['Longitude']],
      popup=final_df.iloc[i]['location'],
      radius= final_df.iloc[i]['ratio']*5000000,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)
m

In [ ]:
m

In [ ]:
bins = list(final_df['daily_vaccinations'].quantile([0, 0.25, 0.5, 0.75, 1]))

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_geo

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    data=final_df,
    columns=["location", "daily_vaccinations"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="Unemployment Rate (%)",
    reset=True,
).add_to(m)

m

In [ ]:
print(state_geo)